In [1]:
import numpy as np
import pandas as pd # for loading data

In [2]:
cd ..

/home/nicktehrany/Documents/Uni/3rd Year/P4/Machine Learning/Project


In [3]:
# Importing our own Functions
from util.helperfunctions import one_hot, one_hot_genres, clean_text, one_hot_reverse

/home/nicktehrany/.local/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/home/nicktehrany/.local/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/home/nicktehrany/.local/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/home/nicktehrany/.local/lib/python3.6/site-packages/tensorflow/python/frame

In [4]:
cd src

/home/nicktehrany/Documents/Uni/3rd Year/P4/Machine Learning/Project/src


In [5]:
train_df = pd.read_csv("train.csv")

## Cleaning the test set lyrics and creating One-Hots for train data's genres and just cleaned lyrics

In [6]:
lyrics, word_list = clean_text(train_df['Lyrics'], 1)

# writing cleaned text back to df
index = 0
for text in lyrics:
    train_df['Lyrics'][index] = text
    index+=1
del lyrics

In [7]:
train_genres = one_hot_genres(train_df['Genre'])

## Replace all the lyrics with their respctive vector of word occurrences to be used as the input layer for the neural network

In [10]:
train_lyrics = np.zeros(shape=(len(train_df),len(word_list)))
for index, value in train_df.iterrows():
    train_lyrics[index] = one_hot(str(train_df['Lyrics'][index]), word_list, len(word_list), 1)
print(train_lyrics.shape)
lyrics_size = len(word_list)

# Cleaning no more used variables
del train_df

(10098, 34260)


## Preparing Validation Set

In [11]:
val_df = pd.read_csv("validation.csv")
lyrics, dummy_list = clean_text(val_df['Lyrics'], 0)

# writing cleaned text back to df
index = 0
for text in lyrics:
    val_df['Lyrics'][index] = text
    index+=1
del lyrics

In [12]:
val_genres = one_hot_genres(val_df['Genre'])

In [13]:
val_lyrics = np.zeros(shape=(len(val_df), lyrics_size))
for index, value in val_df.iterrows():
    val_lyrics[index] = one_hot(str(val_df['Lyrics'][index]), word_list, lyrics_size, 1)
print(val_lyrics.shape)

# Cleaning no more used variables
del val_df

(2142, 34260)


## Cleaning the test set lyrics and creating One-Hots for test data's genres and just cleaned lyrics

In [14]:
test_df = pd.read_csv("test.csv")
lyrics, dummy_list = clean_text(test_df['Lyrics'], 0)

# writing cleaned text back to df
index = 0
for text in lyrics:
    test_df['Lyrics'][index] = text
    index+=1
del lyrics

In [15]:
test_genres = one_hot_genres(test_df['Genre'])

In [16]:
test_lyrics = np.zeros(shape=(len(test_df), lyrics_size))
for index, value in test_df.iterrows():
    test_lyrics[index] = one_hot(str(test_df['Lyrics'][index]), word_list, lyrics_size, 1)
print(test_lyrics.shape)

# Cleaning no more used variables
del test_df

(2160, 34260)


## **Don't rerun the cell below, as this will result in adding dimensions to the sets, which will not work! Can rerun all other cells except this one.**

In [17]:
cnn_lyrics_train = train_lyrics[:, :, None]
cnn_lyrics_val = val_lyrics[:, :, None]
cnn_lyrics_test = test_lyrics[:, :, None]

## Convolutional Neural Network

In [18]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Activation, Conv1D, MaxPool2D, Dropout, Flatten

model = Sequential()
model.add(Conv1D(128, kernel_size=(1), activation='relu', input_shape=(len(word_list),1)))
model.add(Conv1D(32, (1), activation='relu'))
model.add(Dropout(0.8)) # Dropout 25% of the nodes of the previous layer during training
model.add(Flatten())     # Flatten, and add a fully connected layer
model.add(Dense(16, activation='relu')) 
model.add(Dropout(0.2))
model.add(Dense(8, activation='softmax')) # Last layer: 10 class nodes, with dropout
model.summary()

Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv1d (Conv1D)              (None, 34260, 128)        256       
_________________________________________________________________
conv1d_1 (Conv1D)            (None, 34260, 32)         4128      
_________________________________________________________________
dropout (Dropout)            (None, 34260, 32)         0         
_________________________________________________________________
flatten (Flatten)            (None, 1096320)           0         
_________________________________________________________________
dense (Dense)                (None, 16)                17541136  
_________________________________________________________________
dropout_1 (Dropout)          (None, 16)                0         
____

In [22]:
from tensorflow.keras.optimizers import Adam

optimizer = Adam()
model.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['accuracy'])

model.fit(cnn_lyrics_train, train_genres, epochs=5, shuffle=True, batch_size=16, validation_data=(cnn_lyrics_val, val_genres))

Train on 10098 samples, validate on 2142 samples
Epoch 1/5
10098/10098 [==============================] - 337s 33ms/sample - loss: 1.8077 - acc: 0.3215 - val_loss: 1.5380 - val_acc: 0.4734
Epoch 2/5
10098/10098 [==============================] - 399s 39ms/sample - loss: 1.4576 - acc: 0.4629 - val_loss: 1.3846 - val_acc: 0.5093
Epoch 3/5
10098/10098 [==============================] - 313s 31ms/sample - loss: 1.2533 - acc: 0.5364 - val_loss: 1.3331 - val_acc: 0.5271
Epoch 4/5
10098/10098 [==============================] - 314s 31ms/sample - loss: 1.1045 - acc: 0.5948 - val_loss: 1.3407 - val_acc: 0.5266
Epoch 5/5
10098/10098 [==============================] - 316s 31ms/sample - loss: 1.0138 - acc: 0.6284 - val_loss: 1.3663 - val_acc: 0.5294


In [23]:
results = model.evaluate(cnn_lyrics_test, test_genres, batch_size=16)

2160/2160 [==============================] - 17s 8ms/sample - loss: 1.3347 - acc: 0.5417


In [24]:
from sklearn.metrics import confusion_matrix

y_pred=model.predict_classes(cnn_lyrics_test)
cm=confusion_matrix(one_hot_reverse(test_genres),y_pred)
print(cm)

[[ 90  48  11  29  32  30  16  30]
 [ 20  90   1  23  12  64  20  31]
 [  3   1 249   2  27   1   8   0]
 [  8  22   0 170   7   9   1  39]
 [ 15  15  32  11 152  12  26  11]
 [  6  28   4   5   4 165   6  21]
 [ 16  35  23  19  35  30 104  23]
 [  8  38   0  31   7  22  12 150]]


In [25]:
import seaborn as sn
import pandas as pd
import matplotlib.pyplot as plt

genre_list = ['Pop','Rock', 'Hip-Hop/Rap', 'Country', 'R&B/Soul', 'Metal', 'Alternative/Indie', 'Folk']

df_cm = pd.DataFrame(cm, index = genre_list, columns = genre_list)
print(df_cm)
plt.figure(figsize = (10,7))
sn.heatmap(df_cm, annot=True, linewidth=0.5, fmt='g', cmap="BuPu")
b, t = plt.ylim() # discover the values for bottom and top
b += 0.5 # Add 0.5 to the bottom
t -= 0.5 # Subtract 0.5 from the top
plt.ylim(b, t) # update the ylim(bottom, top) values

                   Pop  Rock  Hip-Hop/Rap  Country  R&B/Soul  Metal  \
Pop                 90    48           11       29        32     30   
Rock                20    90            1       23        12     64   
Hip-Hop/Rap          3     1          249        2        27      1   
Country              8    22            0      170         7      9   
R&B/Soul            15    15           32       11       152     12   
Metal                6    28            4        5         4    165   
Alternative/Indie   16    35           23       19        35     30   
Folk                 8    38            0       31         7     22   

                   Alternative/Indie  Folk  
Pop                               16    30  
Rock                              20    31  
Hip-Hop/Rap                        8     0  
Country                            1    39  
R&B/Soul                          26    11  
Metal                              6    21  
Alternative/Indie                104    23  


(8.0, 0.0)